In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install opencv-python==3.4.2.17 opencv-contrib-python==3.4.2.17

     |████████████████████████████████| 25.0MB 166kB/s 
     |████████████████████████████████| 30.6MB 101kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [ ]:
import os
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow

"""TRAIN
"""
train_path = '/content/gdrive/MyDrive/CS331.L22.KHCL/Dataset/DogCatSimple/train/'

"""TEST
"""
test_path = '/content/gdrive/MyDrive/CS331.L22.KHCL/Dataset/DogCatSimple/test/'

In [ ]:
test_lb =   {
  "1": "dog",   "2": "dog",   "3": "dog",   "4": "dog",   "5": "cat",
  "6": "cat",   "7": "cat",   "8": "cat",   "9": "cat",   "10": "cat",
  "11": "cat",  "12": "dog",  "13": "cat",  "14": "cat",  "15": "cat", 
  "16": "cat",  "17": "dog",  "18": "dog",  "19": "cat",  "20": "cat",
}

## Indexing/labeling each image and computer descriptors

In [ ]:
"""INDEX FILES
"""
def index_files(file_path):
    id_file_path = dict()
    img_path = []
    for image_name in os.listdir(file_path):
        img_path.append(os.path.join(file_path + image_name))
    for id, image in enumerate(img_path):
        id_file_path[id] = image
    
    return id_file_path


"""GET LABELS
"""
def get_labels(id_file_path):
    id_and_label_dict = dict()
    for id, file_path in id_file_path.items():
        file_name = os.path.basename(file_path)
        id_and_label_dict[id] = file_name[0:3]

    return id_and_label_dict


"""COMPUTE DESCRIPTORS
"""
sift_descriptor = cv2.xfeatures2d.SIFT_create()
from skimage.feature import hog
from skimage.transform import resize
 

def get_descriptor(id_and_label_dict, mode=None):
    descriptor_id_label = dict()
    for id, path in id_and_label_dict.items():
        try:
            image = cv2.imread(path)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            if mode == 'SIFT':
                (kps, descriptors) = sift_descriptor.detectAndCompute(image, None)
                if descriptors is not None:
                    descriptor_id_label[id] = descriptors
            if mode == 'HOG':
                resized_img = resize(gray_image, (256, 256))
                descriptors, _ =  hog(resized_img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)  
                if descriptors is not None:
                    descriptor_id_label[id] = descriptors
        except:
            continue

    return descriptor_id_label

In [ ]:
# EXECUTING
# to do: choose 'sift' or 'hog' feature extractor mode.
id_train_path = index_files(train_path)
id_test_path = index_files(test_path)

id_and_train_dict = get_labels(id_train_path)
id_and_test_dict = get_labels(id_test_path)

descriptor_id_train = get_descriptor(id_train_path, mode='SIFT')
descriptor_id_test = get_descriptor(id_test_path, mode='SIFT')

## Create BAG OF VISUAL WORDS

In [ ]:
def stack_descriptors(descriptor_of_imgs):
    result = descriptor_of_imgs[0]
    for des in descriptor_of_imgs[1:]:
        result = np.vstack((result, des))
    return result

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 60)

## EXECUTING
stacked_descriptor_train = stack_descriptors(list(descriptor_id_train.values()))
stacked_descriptor_test = stack_descriptors(list(descriptor_id_test.values()))

vocab_descriptor = np.vstack((stacked_descriptor_train, stacked_descriptor_test))

kmeans.fit(vocab_descriptor)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=60, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

# Build histogram

In [ ]:
"""BUILD HISTOGRAM
"""
def build_his(descriptors, kmeans):
    histogram = np.zeros(len(kmeans.cluster_centers_))
    cluster_result =  kmeans.predict(descriptors)
    for i in cluster_result:
        histogram[i] += 1.0

    return histogram

def build_his_4all(descriptor_id, kmeans):
    his_id_list = dict()
    for id, descriptor in descriptor_id.items():     
        if (descriptor is not None):
            histogram = build_his(descriptor, kmeans)
            his_id_list[id] = histogram
  
    return his_id_list


"""NORMALIZE
"""
def normalize(vectors, descriptors):
  for id, vector in vectors.items():
    vectors[id] = vectors[id] / len(descriptors[id])

In [ ]:
his_id_train = build_his_4all(descriptor_id_train, kmeans)
his_id_test = build_his_4all(descriptor_id_test, kmeans)

normalize(his_id_train, descriptor_id_train)
normalize(his_id_test, descriptor_id_test)

## Separate to each label and train/test feature

In [ ]:
def his_n_label_train(histograms, labels):
    histogram_list = list()
    label_list = list()
    for id, label in labels.items():
        histogram_list.append(histograms[id])
        label_list.append(label)
  
    return histogram_list, label_list

In [ ]:
def his_n_label_test(histograms, id_and_file_paths, true_label):
    histogram_list = list()
    label_list = list()

    for id, file_path in id_and_file_paths.items():
        file_name = os.path.basename(file_path)
        index = os.path.splitext(file_name)[0]
        label = true_label[index]

        histogram_list.append(histograms[id])
        label_list.append(label)
        

    return histogram_list, label_list

In [ ]:
feature_train, label_train = his_n_label_train(his_id_train, id_and_train_dict)
feature_test, label_test = his_n_label_test(his_id_test, id_and_test_dict, test_lb)

## Train and predict!

In [ ]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(feature_train, label_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
prediction = svm.predict(feature_test)

In [ ]:
"""SIFT DESCRIPTOR RESULTS.
"""
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

"""Results of SIFT descriptor.
"""
print('Accuracy score: ', accuracy_score(label_test, prediction))
print('Pre score: ', precision_score(label_test, prediction, pos_label='cat'))
print('Rec score: ', recall_score(label_test, prediction, pos_label='cat'))
print('F1 score: ', f1_score(label_test, prediction, pos_label='cat'))

Accuracy score:  0.65
Pre score:  0.7142857142857143
Rec score:  0.7692307692307693
F1 score:  0.7407407407407408
